In [1]:
import xarray as xr
import postmit as pm

In [2]:
#path_to_data = "/path/to/data/"
#path_to_input = "/path/to/input/"
path_to_data = "/home/jan/Data/MITgcm_test_data/nc_Diags/"
path_to_input = "/home/jan/Data/MITgcm_test_data/input/"

### Load some data
Here we load data that has already been converted to `netCDF` format with [`convert.convert2nc`](https://github.com/jk-rieck/postmit/blob/main/examples/example_convert.ipynb).

In [9]:
#ds = xr.open_dataset(path_to_data + "data_1m_0222.nc")
ds = xr.merge([xr.open_dataset(path_to_data + "2D_diags_1m_0222.nc"), xr.open_dataset(path_to_data + "3D_diags_1m_0222.nc"), 
               xr.open_dataset(path_to_data + "ice_diags_1m_0222.nc"), xr.open_dataset(path_to_data + "layers_diags_1m_0222.nc")])

### Apply some checks
Here we will apply all checks with [`checks.apply_all_checks`](https://github.com/jk-rieck/postmit/blob/main/postmit/checks.py#L137).  
See [example_checks.ipynb](https://github.com/jk-rieck/postmit/blob/main/examples/example_checks.ipynb) for more details.

In [10]:
ds = pm.checks.apply_all_checks(ds, path_to_input)

### The quantities that can be computed:

| variable          | description                                                  | grid               | units      | function          | depends on                          |
| :-                | :-                                                           | :-                 |            | :-                | :-                                  |
| VORT              | vorticity                                                    | (time, Z, YG, XG)  | 1/s        | vort              | UVEL, VVEL, `xgcm`                  |
| RosNum            | Rossby number                                                | (time, Z, YG, XG)  |            | rossby_num        | VORT, fU                            |
| UTRANS            | mean zonal transport integrated over Y and Z                 | (time)             | m^3/s      | transports        | UVEL, `xgcm`                        |
| VTRANS            | mean meridional transport integrated over X and Z            | (time)             | m^3/s      | transports        | VVEL, `xgcm`                        |
| UVELbot           | zonal velocity at the bottom (deepest ocean grid cell)       | (time, Z, YC, XG)  | m/s        | transports        | UVEL                                |
| VVELbot           | meridional velocity at the bottom (deepest ocean grid cell)  | (time, Z, YG, XC)  | m/s        | transports        | VVEL                                |
| UTRANSbaro        | mean zonal baroclinic transport integrated over Y and Z      | (time)             | m^3/s      | transports        | `xgcm`, UVELbot                     |
| VTRANSbaro        | mean meridional baroclinic transport integrated over X and Z | (time)             | m^3/s      | transports        | `xgcm`, VVELbot                     |
| SIG0              | potential density referenced to the surface (0 dbar)         | (time, Z, YC, XC)  | kg/m^3     | sig0              | SALT, THETA, `MITgcmutils`          |
| SIGi              | potential density referenced to `i * 1000` dbar (`i < 10`)   | (time, Z, YC, XC)  | kg/m^3     | sigi              | SALT, THETA, `MITgcmutils`          | 
||||total_MOC||
||||residual_MOC||
||||eddy_MOC||
| gravity           | gravity used in the simulation (default=9.81)                |                    | m/s^2      | get_const         |                                     |
| rhonil            | rhonil used in the simulation (default=999.8                 |                    | kg/m^3     | get_const         |                                     |
| rhoconst          | rhoconst used in the simulation (default=rhonil)             |                    | kg/m^3     | get_const         |                                     |
| HeatCapacity_cp   | HeatCapacity_cp used in the simulation (default=3994)        |                    | J/kg/K     | get_const         |                                     |
| ups               | scale factor to convert model salinity to preformed          |                    |            | get_const         |                                     |
| fF                | coriolis parameter on t grid                                 | (time, Z, YC, XC)  | 1/s        | get_const         |                                     |
| fG                | coriolis parameter on v grid                                 | (time, Z, YG, XC)  | 1/s        | get_const         |                                     |
| fC                | coriolis parameter on u grid                                 | (time, Z, YC, XG)  | 1/s        | get_const         |                                     |
| fU                | coriolis parameter on f grid                                 | (time, Z, YG, XG)  | 1/s        | get_const         |                                     |
| SEAICE_waterDrag  | drag coefficient for stress between ice and water            |                    |            | get_const         | `SEAICE` package                    |
| BUOY              | buoyancy                                                     | (time, Z, YC, XC)  | m/s^2      | buoy              | RHOAnoma, gravity, rhoconst         |
| PRESS             | pressure                                                     | (time, Z, YC, XC)  | dbar       | press             | PHIHYD, rhonil                      |
| DENS              | in-situ density                                              | (time, Z, YC, XC)  | kg/m^3     | dens              | SALT, THETA, PRESS, `MITgcmutils`   |
| SA                | absolute salinity                                            | (time, Z, YC, XC)  | g/kg       | SA                | `gsw`, SALT, ups, PRESS, lonF, latF |
| alpha             | thermal expansion coefficient                                | (time, Z, YC, XC)  | 1/K        | alpha             | `gsw`, SA, THETA, PRESS             |
| beta              | haline contraction coefficient                               | (time, Z, YC, XC)  | kg/g       | beta              | `gsw`, SA, THETA, PRESS             |
| BFlx_SURF         | surface buoyancy flux                                        | (time, Z, YC, XC)  |            | surface_buoy_flux | gravity, rhoconst, HeatCapacity_Cp, SA, alpha, beta, oceQnet, oceFWflx |
| WVELEk            | Ekman vertical velocity                                      | (time, Z, YG, XG)  | m/s        | w_ekman           | rhoconst, `xgcm`, VORT, EXFtaux, EXFtauy |
| SIOtaux           | zonal sea ice-ocean stress                                   | (time, Z, YC, XG)  | N/m^2      | ice_ocean_stress  | rhoconst, SEAICE_waterDrag, SIuice, UVEL, SIheff, SIarea |
| SIOtauy           | meridional sea ice-ocean stress                              | (time, Z, YG, XC)  | N/m^2      | ice_ocean_stress  | rhoconst, SEAICE_waterDrag, SIvice, VVEL, SIheff, SIarea |

In [12]:
ds

<xarray.Dataset>
Dimensions:       (YC: 300, XC: 250, time: 12, Z: 55, Zp1: 56, XG: 250,
                   YG: 300, Zl: 55, Zu: 55, layer_center: 50)
Coordinates: (12/37)
    Depth         (YC, XC) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    PHrefC        (Z) float32 dask.array<chunksize=(55,), meta=np.ndarray>
    PHrefF        (Zp1) float32 dask.array<chunksize=(56,), meta=np.ndarray>
  * XC            (XC) float32 5e+03 1.5e+04 2.5e+04 ... 2.485e+06 2.495e+06
  * XG            (XG) float32 0.0 1e+04 2e+04 ... 2.47e+06 2.48e+06 2.49e+06
  * YC            (YC) float32 5e+03 1.5e+04 2.5e+04 ... 2.985e+06 2.995e+06
    ...            ...
    rAz           (YG, XG) float32 dask.array<chunksize=(300, 250), meta=np.ndarray>
    rhoRef        (Z) float32 dask.array<chunksize=(55,), meta=np.ndarray>
  * time          (time) object 0222-01-15 00:00:00 ... 0222-12-15 00:00:00
  * layer_center  (layer_center) float64 23.25 23.75 24.25 ... 28.35 28.45 28.75
    drW           (Z, YC, XG) float32 dask.array<chunksize=(55, 300, 250), meta=np.ndarray>
    drS           (Z, YG, XC) float32 dask.array<chunksize=(55, 300, 250), meta=np.ndarray>
Data variables: (12/44)
    ETAN          (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFempmr      (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFevap       (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFhl         (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFhs         (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    EXFlwnet      (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    ...            ...
    SItices       (time, YC, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    SIuice        (time, YC, XG) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    SIvice        (time, YG, XC) float32 dask.array<chunksize=(12, 300, 250), meta=np.ndarray>
    LaHs1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
    LaVH1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
    LaVa1RHO      (time, layer_center, YG, XC) float32 dask.array<chunksize=(12, 50, 300, 250), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    title:        netCDF wrapper of MITgcm MDS binary data
    source:       MITgcm
    history:      Tue Dec 21 16:44:29 2021: ncks -O -4 -L 1 2D_diags_1m_0222....
    NCO:          netCDF Operators version 4.9.5 (Homepage = http://nco.sf.ne...